In [88]:
# !python -m pip install --upgrade pip
# !pip install --upgrade google-cloud-bigquery
# !pip install --upgrade google-cloud-storage
# !pip install --upgrade google-cloud-pubsub
# gcloud init
# gcloud auth application-default login
# gcloud config set project 'project-id'
# !gcloud pubsub topics create my-topic
# !gcloud pubsub subscriptions create my-sub --topic my-topic
# !gcloud iam service-accounts create SERVICE_ACCOUNT_NAME
# !gcloud projects add-iam-policy-binding PROJECT_ID --member="serviceAccount:SERVICE_ACCOUNT_NAME@PROJECT_ID.iam.gserviceaccount.com" --role=ROLE

In [90]:
# !gcloud iam service-accounts create pubsub
# !gcloud projects add-iam-policy-binding triple-baton-382908 --member="serviceAccount:pubsub@triple-baton-382908.iam.gserviceaccount.com" --role=roles/pubsub.admin

In [20]:
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import pubsub_v1
import os
import google.auth

In [ ]:
PROJECT_ID = "triple-baton-382908"
bq_credentials, project = google.auth.default()
bq_client = bigquery.Client(project=PROJECT_ID)
storage_client = storage.Client(project=PROJECT_ID)

# Create topic and subscription

In [ ]:
publisher = pubsub_v1.PublisherClient()

project_id = "your-project-id"
topic_id = "your-topic-id"
subscription_id = "your-subscriber-id"

publisher = pubsub_v1.PublisherClient()
subscriber = pubsub_v1.SubscriberClient()

topic_path = publisher.topic_path(project_id, topic_id)
subscription_path = subscriber.subscription_path(project_id, subscription_id)
topic = publisher.create_topic(request={"name": topic_path})


with subscriber:
    subscription = subscriber.create_subscription(
        request={"name": subscription_path, "topic": topic_path}
    )

print(f"Subscription created: {subscription}")

# add schema to a topic

In [ ]:
from google.api_core.exceptions import AlreadyExists, InvalidArgument
from google.cloud.pubsub import PublisherClient, SchemaServiceClient
from google.pubsub_v1.types import Encoding

# TODO(developer): Replace these variables before running the sample.
# project_id = "your-project-id"
# topic_id = "your-topic-id"
# schema_id = "your-schema-id"
# first_revision_id = "your-revision-id"
# last_revision_id = "your-revision-id"
# Choose either BINARY or JSON as valid message encoding in this topic.
# message_encoding = "BINARY"

publisher_client = PublisherClient()
topic_path = publisher_client.topic_path(project_id, topic_id)

schema_client = SchemaServiceClient()
schema_path = schema_client.schema_path(project_id, schema_id)

if message_encoding == "BINARY":
    encoding = Encoding.BINARY
elif message_encoding == "JSON":
    encoding = Encoding.JSON
else:
    encoding = Encoding.ENCODING_UNSPECIFIED
    
try:
    response = publisher_client.create_topic(
        request={
            "name": topic_path,
            "schema_settings": {
                "schema": schema_path,
                "encoding": encoding,
                "first_revision_id": first_revision_id,
                "last_revision_id": last_revision_id,
            },
        }
    )
    print(f"Created a topic:\n{response}")

except AlreadyExists:
    print(f"{topic_id} already exists.")
except InvalidArgument:
    print("Please choose either BINARY or JSON as a valid message encoding type.")

In [ ]:
!cat /Users/{username}/.config/gcloud/application_default_credentials.json


In [74]:
# credentials_path = ""
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
publisher = pubsub_v1.PublisherClient()
publisher.create_topic("projects/triple-baton-382908/topics/my-topic")
topic_path = publisher.topic_path(PROJECT_ID, "my-topic")

for i in range(2):
    data = "Message number {}".format(i)
    data = data.encode("utf-8")
    attributes = {"origin": "python-sample"}
    future = publisher.publish(topic_path, data=data, **attributes)
    print("published {} of message ID {}".format(data, future.result()))

/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


published b'Message number 0' of message ID 8466831988385189
published b'Message number 1' of message ID 8466837745448942


In [64]:


# credentials_path = ""
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path




/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [75]:
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(PROJECT_ID, "my-sub")

streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print("Listening for messages on {}..\n".format(subscription_path))

with subscriber:
    try:
        streaming_pull_future.result(timeout=5)
    except TimeoutError:
        streaming_pull_future.cancel()

/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Listening for messages on projects/triple-baton-382908/subscriptions/my-sub..

Received message: Message {
  data: b'Message number 0'
  ordering_key: ''
  attributes: {
    "origin": "python-sample"
  }
}
data: b'Message number 0'
attributes:
origin: python-sample
Received message: Message {
  data: b'Message number 1'
  ordering_key: ''
  attributes: {
    "origin": "python-sample"
  }
}
data: b'Message number 1'
attributes:
origin: python-sample


In [67]:
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)

Thread-ConsumeBidirectionalStream caught unexpected exception Cannot invoke RPC: Channel closed! and will exit.
Traceback (most recent call last):
  File "/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/api_core/bidi.py", line 636, in _thread_main
    self._bidi_rpc.open()
  File "/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/api_core/bidi.py", line 279, in open
    call = self._start_rpc(iter(request_generator), metadata=self._rpc_metadata)
  File "/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/pubsub_v1/services/subscriber/client.py", line 1589, in streaming_pull
    response = rpc(
  File "/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/api_core/gapic_v1/method.py", line 113, in __call__
    return wrapped_func(*args, **kwargs)
  File "/Users/hardey/opt/anaconda3/envs/bigdata/lib/python3.9/site-packages/google/api_core/retry.py", line 349, in retry_wrapped_func
    retu

In [ ]:

bigquery_table_id = "your-project.your-dataset.your-table"
bigquery_config = pubsub_v1.types.BigQueryConfig(
    table=bigquery_table_id, write_metadata=True
)

# Wrap the subscriber in a 'with' block to automatically call close() to
# close the underlying gRPC channel when done.
with subscriber:
    subscription = subscriber.create_subscription(
        request={
            "name": subscription_path,
            "topic": topic_path,
            "bigquery_config": bigquery_config,
        }
    )
